In [ ]:
import os 
os.environ["HF_HOME"] = "/your/path/hf_cache"
from datasets import load_dataset 

    

In [ ]:

eval_data = load_dataset('json', 
                         data_files="/your/path/visual7w/dataset.json", 
                         split="train")

In [ ]:
from PIL import Image
idx = 0
# print(eval_data[idx].keys()) 
# # dict_keys(['images', 'version', 'dataset'])
# print(eval_data[idx]["images"].keys()) 
# dict_keys(['filename', 'image_id', 'qa_pairs', 'split'])
for qa_pair in eval_data[idx]["images"]['qa_pairs']:
    print(qa_pair['question'], qa_pair['answer'])

In [ ]:
count = 0
sunnycount = 0
rainycount = 0
snowycount = 0
cloudycount = 0
duskcount = 0
stormycount = 0
coldcount = 0
warmcount = 0
all_other_answers = []
# There are instances with multiple weather queries
for item in eval_data:
    if item["images"]["split"] != 'val': continue
    for qa_pair in item["images"]["qa_pairs"]:
        if "weather" in qa_pair["question"]:
            if "sunny" in qa_pair["answer"].lower() or "clear" in qa_pair["answer"].lower() or "sunshine" in qa_pair["answer"].lower(): 
                sunnycount += 1
            elif "cloudy" in qa_pair["answer"].lower() or "overcast" in qa_pair["answer"].lower() or "misty" in qa_pair["answer"].lower(): 
                cloudycount += 1
            elif "rainy" in qa_pair["answer"].lower() or "drizzly" in qa_pair["answer"].lower() or "rain" in qa_pair["answer"].lower():
                rainycount += 1
            elif "snow" in qa_pair["answer"].lower(): snowycount += 1
            elif "stormy" in qa_pair["answer"].lower(): stormycount += 1
            elif "warm" in qa_pair["answer"].lower(): 
                print(qa_pair['question'], qa_pair['answer'])
                warmcount += 1
            elif "cold" in qa_pair["answer"].lower() or "chilly" in qa_pair["answer"].lower(): coldcount += 1
            else:
                # print(qa_pair['question'], qa_pair['answer'])
                all_other_answers.append(qa_pair['answer'])
                continue
            count += 1
        elif "cloudy" in qa_pair["answer"].lower():
            cloudycount += 1
            count += 1
        

print(count, sunnycount, rainycount, snowycount, cloudycount, stormycount, warmcount, coldcount)


In [ ]:
from wordcloud import WordCloud

# Word cloud
import matplotlib.pyplot as plt

# Generate a word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(all_other_answers))

# Display the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# Time: day, night/dark, evening, afternoon
categories = {"sunny":[], "cloudy":[], "rainy":[], "snowy":[], "stormy": [], "warm": [], "cold": [],
              "clear":[], "misty": [], "overcast":[], "drizzly": [], "chilly": []}
for item in eval_data:
    if item["images"]["split"] != 'val': continue
    gt = None
    for qa_pair in item["images"]["qa_pairs"]:
        if "weather" in qa_pair["question"]:
            if "sunny" in qa_pair["answer"].lower() or "sunshine" in qa_pair["answer"].lower(): 
                gt = "sunny"
            elif "clear" in qa_pair["answer"].lower():
                gt = "clear"
            elif "cloudy" in qa_pair["answer"].lower(): 
                gt = "cloudy"
            elif "overcast" in qa_pair["answer"].lower():
                gt = "overcast" 
            elif "misty" in qa_pair["answer"].lower(): 
                gt = "misty"
            elif "rainy" in qa_pair["answer"].lower() or "rain" in qa_pair["answer"].lower():
                gt = "rainy"
            elif "drizzly" in qa_pair["answer"].lower():
                gt = "drizzly"
            elif "snow" in qa_pair["answer"].lower(): gt = "snowy"
            elif "stormy" in qa_pair["answer"].lower(): gt = "stormy"
            elif "warm" in qa_pair["answer"].lower(): gt = "warm"
            elif "cold" in qa_pair["answer"].lower(): gt = "cold"
            elif "chilly" in qa_pair["answer"].lower(): gt = "chilly"
        elif "cloudy" in qa_pair["answer"].lower():
            gt = "cloudy"

    if gt != None: 
        categories[gt].append(item['images']['filename'])

In [ ]:
import random

new_data = []
exclusive = {"sunny": ["cloudy", "misty", "overcast", "rainy", "drizzly", "snowy", "stormy", "cold", "chilly"], 
             "clear":  ["cloudy", "misty", "overcast", "rainy", "drizzly", "snowy", "stormy", "cold", "chilly"], 
             "misty":  ["sunny", "clear", "snowy"], 
             "overcast": ["sunny", "clear", "snowy"],
             "cloudy": ["sunny", "clear", "snowy"],
             "rainy": ["sunny", "clear", "warm", "snowy"], 
             "drizzly": ["sunny", "clear", "warm", "snowy"], 
             "stormy": ["sunny", "clear", "warm", "snowy"],
             "snowy": ["sunny", "clear", "rainy", "drizzly", "stormy", "warm"],
             "warm": ["rainy", "stormy", "snowy", "cold", "chilly", "overcast", "misty"],
             "cold": ["sunny", "clear", "warm"],
             "chilly": ["sunny", "clear", "warm"]}
for label in categories:
    for data in categories[label]:
        new_item = {}
        if label == "warm":
            continue # We exclude warm here
        if label == "overcast":
            new_item['qry_text'] = f"Find me an everyday image that is taken in an {label} day.\n"
        else:
            new_item['qry_text'] = f"Find me an everyday image that is taken in a {label} day.\n" 
        new_item['qry_img_path'] = ''
        new_item['tgt_text'] = "<|image_1|> Represent the given image."
        new_item['tgt_img_path'] = [data]
        all_paths = []
        for place in categories:
            if place in exclusive[label]:
                all_paths.extend(categories[place])
        new_item['tgt_img_path'].extend(random.sample(all_paths, 99))
        new_data.append(new_item)


In [ ]:
import json

with open('Visual7W_weather_retrieval.json', 'w') as f:
    json.dump(new_data, f, indent=4)

In [ ]:
# Test whether we can load it using load_dataset
new_eval_data = load_dataset('json', 
                      data_files='Visual7W_weather_retrieval.json',
                      split="train")

In [ ]:
idx = 1
print(new_eval_data[idx].keys())
print(new_eval_data[idx]['qry_text'])
img = Image.open("/your/path/visual7w/images/"+new_eval_data[idx]['tgt_img_path'][60]).convert("RGB")
img